# Tick Test Analysis 

In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import datetime

matplotlib.rcParams[ 'figure.figsize' ] = ( 14, 8 )

def show_time( label_string ):
    ts = time.time()
    st = datetime.datetime.fromtimestamp( ts ).strftime( '%Y-%m-%d %H:%M:%S:%f' )
    print( label_string + ' : ' + st )

In [ ]:
# load our taq file
taq = pd.read_pickle( 'TWTR_taq_20190102.gz' )

# get all the trades for later too
market_trades = pd.read_pickle( 'TWTR_trades_201901.gz' )
market_trades = market_trades.between_time( '09:30:00', '16:00:00' )

quotes = pd.read_pickle( 'TWTR_quotes_201901.gz' )
quotes = quotes.between_time( '09:30:00', '16:00:00' )

## Spreads

In [ ]:
# todo

In [ ]:
avg_spread = ( quotes[ 'ask_price' ] - quotes[ 'bid_price' ] ).mean()

In [ ]:
spreads = quotes[ 'ask_price' ] - quotes[ 'bid_price' ]

In [ ]:
# trim our outliers
spreads = spreads[ spreads <= 5 ]
spreads = spreads[ spreads > 0 ]
avg_spread = spreads.mean()

In [ ]:
avg_spread

In [ ]:
fig, ax1 = plt.subplots (figsize=(16,9))
ax1.hist( spreads, bins = np.arange( 0.01, 0.20, 0.01 ), rwidth = .5, log = False )
plt.show()

## Tick Test and Autocorrelation

In [ ]:
len(market_trades)

In [ ]:
# tick test to do up/down
# use shift() to calculate tick 

# grab one day
trading_day = market_trades['2019-01-02']

ticks = np.sign( trading_day[ 'trade_price' ] - trading_day[ 'trade_price' ].shift( 1 ) )
ticks_adj = ticks.copy()

In [ ]:
ticks.head(10)

In [ ]:
# but we want zeros to be directional from the previous value so let's fill in
# I'm going to be lazy and loop this

prev_item = 0
for index, item in ticks.iteritems():
    if not pd.isnull( item ):
        if item == 0:
            ticks_adj[index] = prev_item
        else:   
            # a nonzero value, set our counter
            prev_item = item
        

In [ ]:
ticks_adj.head(10)

In [ ]:
# Calculate autocorrelation for n lags 
fig, ax1 = plt.subplots(figsize=(16,9))
for i in np.arange( 1, 100 ):
    ax1.bar( i, ticks_adj.autocorr( lag = i ), color = 'lightblue' )
ax1.set_title( 'Autocorrelation of tick test data by lag' )
ax1.set_ylabel( 'Autocorrelation' )
ax1.set_xlabel( 'Lag' )
plt.show()